In [98]:
# dictionary mapping gene names to list of last included exons
five_prime_genes = {
    # MGA exon 22, NUTM1 exon 3, source: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6318763/
    "MGA": [22], 
    
    # BRD4 exons 10 & 11, sources:
    # - https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5378225/
    # - https://aacrjournals.org/cancerres/article/63/2/304/510577/BRD4-NUT-Fusion-OncogeneA-Novel-Mechanism-in
    #
    # - exon 15, source: https://www.nature.com/articles/onc2012487
    "BRD4": [10, 11, 12, 13, 14, 15],
    
    # BRD3 exon 9, NUTM1 exon 2, source: https://www.nature.com/articles/1210852
    "BRD3": [9],
    
    
    # MXD4 exon 5, NUTM1 exons 2 & 3, sources:
    # - https://pubmed.ncbi.nlm.nih.gov/30338611/
    # - https://www.nature.com/articles/s41379-021-00792-z
    "MXD4": [5],
    
    # CIC exons 16-20, NUTM1 exons 2-5, source: https://hal.archives-ouvertes.fr/hal-01927040
    "CIC": [16, 17, 18, 19, 20],
    
    # SLC12A6 exon 2, NUTM1 exon 3, source: https://www.haematologica.org/article/view/9099
    "SLC12A6": [2],
    
    # YAP1 exon 3, NUTM1 exon 2, source: https://link.springer.com/article/10.1007/s12105-020-01173-9
    "YAP1": [3],
}

# * even though the CIC paper claims there are fusions on exons 4 & 5, I don't see those in any other papers
# * OK, found one reference with BRD4 exon 11 fused onto NUTM1 exon 3, both of which have end/start phases of 1
#   source: 
# * Oy, found that older references missed what is now considered exon 1, so adding exon 6 which was the old exon 5
NUTM1_start_exons = [2, 3, 4, 5, 6] 

In [99]:
from pyensembl import ensembl_grch38

In [100]:
name_to_gene = {}
for name in set(five_prime_genes.keys()).union({"NUTM1"}):
    genes = ensembl_grch38.genes_by_name(name)
    assert len(genes) == 1
    name_to_gene[name] = genes[0]

In [101]:
def transcript_key(t):
    return (t.complete, -t.support_level if t.support_level else 100, len(t.protein_sequence) if t.protein_sequence else 0)

def pick_best_transcript(ts):
    return sorted(ts, key=transcript_key)[-1]

In [102]:
canonical_transcripts = {"NUTM1": "NUTM1-203"}

name_to_transcript = {}

for (name, gene) in name_to_gene.items():
    if name in canonical_transcripts:
        transcript_name = canonical_transcripts[name]
        transcript = ensembl_grch38.transcripts_by_name(transcript_name)[0]
    else:
        transcript = pick_best_transcript(gene.transcripts)
    name_to_transcript[name] = transcript

In [103]:
name_to_transcript

{'CIC': Transcript(transcript_id='ENST00000681038', transcript_name='CIC-209', gene_id='ENSG00000079432', biotype='protein_coding', contig='19', start=42269252, end=42295796, strand='+', genome='GRCh38'),
 'SLC12A6': Transcript(transcript_id='ENST00000676379', transcript_name='SLC12A6-221', gene_id='ENSG00000140199', biotype='protein_coding', contig='15', start=34230036, end=34337462, strand='-', genome='GRCh38'),
 'BRD4': Transcript(transcript_id='ENST00000679869', transcript_name='BRD4-213', gene_id='ENSG00000141867', biotype='protein_coding', contig='19', start=15235519, end=15332539, strand='-', genome='GRCh38'),
 'YAP1': Transcript(transcript_id='ENST00000282441', transcript_name='YAP1-201', gene_id='ENSG00000137693', biotype='protein_coding', contig='11', start=102110447, end=102233424, strand='+', genome='GRCh38'),
 'BRD3': Transcript(transcript_id='ENST00000303407', transcript_name='BRD3-201', gene_id='ENSG00000169925', biotype='protein_coding', contig='9', start=134030305, end

In [104]:
name_to_coding_exon_lengths = {}
for name, transcript in name_to_transcript.items():
    exon_coords = transcript.coding_sequence_position_ranges
    if transcript.strand == "-":
        exons = reversed([(end, start) for (start, end) in exon_coords])
    name_to_coding_exon_lengths[name] = [end - start + 1 for (start, end) in exon_coords]
        

In [105]:
name_to_coding_exon_lengths

{'CIC': [2794,
  150,
  235,
  130,
  183,
  166,
  203,
  226,
  104,
  1234,
  188,
  122,
  167,
  294,
  326,
  245,
  155,
  132,
  132,
  365],
 'SLC12A6': [271,
  45,
  95,
  132,
  147,
  55,
  131,
  242,
  215,
  159,
  99,
  58,
  175,
  119,
  99,
  120,
  105,
  169,
  196,
  170,
  132,
  108,
  185,
  134,
  32,
  33],
 'BRD4': [285,
  138,
  136,
  290,
  363,
  129,
  210,
  200,
  296,
  111,
  53,
  370,
  588,
  113,
  163,
  131,
  206,
  238,
  66],
 'YAP1': [321, 251, 116, 114, 182, 48, 131, 113, 236],
 'BRD3': [213, 138, 148, 215, 372, 129, 192, 236, 293, 129, 113],
 'MXD4': [64, 100, 30, 115, 163, 155],
 'MGA': [1064,
  949,
  79,
  96,
  132,
  105,
  659,
  346,
  227,
  186,
  73,
  518,
  151,
  291,
  1505,
  131,
  52,
  207,
  112,
  234,
  177,
  1274],
 'NUTM1': [6, 94, 709, 129, 137, 287, 117, 2001]}

In [106]:
name_to_5prime_utr_exons_count = {}
for name, t in name_to_transcript.items():
    
    if t.strand == "+":
        start_codon_pos = min(t.start_codon_positions)
        count = sum([
            end < start_codon_pos
            for (_, end) in t.exon_intervals])
    else:
        start_codon_pos = max(t.start_codon_positions)
        count = sum([
            start > start_codon_pos
            for (start, end) in t.exon_intervals])
    name_to_5prime_utr_exons_count[name] = count
name_to_5prime_utr_exons_count

{'CIC': 1,
 'SLC12A6': 0,
 'BRD4': 1,
 'YAP1': 0,
 'BRD3': 1,
 'MXD4': 0,
 'MGA': 1,
 'NUTM1': 0}

In [107]:

name_and_exon_to_cds_length = {}

for name, exon_numbers in list(five_prime_genes.items()) + [("NUTM1", NUTM1_start_exons)]:
    print(name, exon_numbers) 
    exon_numbers = [min(exon_numbers) - 1] + exon_numbers
    num_utr_exons = name_to_5prime_utr_exons_count[name]
    for exon_number in exon_numbers:
        cds_length = sum(name_to_coding_exon_lengths[name][:(exon_number - num_utr_exons)])
        print("-- %s %d: %d (%d)" % (name, exon_number, cds_length, cds_length % 3))
        name_and_exon_to_cds_length[(name, exon_number)] = cds_length

MGA [22]
-- MGA 21: 7117 (1)
-- MGA 22: 7294 (1)
BRD4 [10, 11, 12, 13, 14, 15]
-- BRD4 9: 1751 (2)
-- BRD4 10: 2047 (1)
-- BRD4 11: 2158 (1)
-- BRD4 12: 2211 (0)
-- BRD4 13: 2581 (1)
-- BRD4 14: 3169 (1)
-- BRD4 15: 3282 (0)
BRD3 [9]
-- BRD3 8: 1407 (0)
-- BRD3 9: 1643 (2)
MXD4 [5]
-- MXD4 4: 309 (0)
-- MXD4 5: 472 (1)
CIC [16, 17, 18, 19, 20]
-- CIC 15: 6196 (1)
-- CIC 16: 6522 (0)
-- CIC 17: 6767 (2)
-- CIC 18: 6922 (1)
-- CIC 19: 7054 (1)
-- CIC 20: 7186 (1)
SLC12A6 [2]
-- SLC12A6 1: 271 (1)
-- SLC12A6 2: 316 (1)
YAP1 [3]
-- YAP1 2: 572 (2)
-- YAP1 3: 688 (1)
NUTM1 [2, 3, 4, 5]
-- NUTM1 1: 6 (0)
-- NUTM1 2: 100 (1)
-- NUTM1 3: 809 (2)
-- NUTM1 4: 938 (2)
-- NUTM1 5: 1075 (1)


In [109]:
name_and_exon_start_phase = {}
name_and_exon_end_phase = {}
for name, exon_numbers in list(five_prime_genes.items()) + [("NUTM1", NUTM1_start_exons)]:
    for exon_number in exon_numbers:
        cds_length_prev = name_and_exon_to_cds_length[(name, exon_number - 1)]
        cds_length = name_and_exon_to_cds_length[(name, exon_number)]
        start_phase = cds_length_prev % 3
        end_phase = cds_length % 3
        name_and_exon_start_phase[(name, exon_number)] = start_phase
        name_and_exon_end_phase[(name, exon_number)] = end_phase
        print("%s exon %d, start phase %d, end phase %d" % (name, exon_number, start_phase, end_phase))
        

MGA exon 22, start phase 1, end phase 1
BRD4 exon 10, start phase 2, end phase 1
BRD4 exon 11, start phase 1, end phase 1
BRD4 exon 12, start phase 1, end phase 0
BRD4 exon 13, start phase 0, end phase 1
BRD4 exon 14, start phase 1, end phase 1
BRD4 exon 15, start phase 1, end phase 0
BRD3 exon 9, start phase 0, end phase 2
MXD4 exon 5, start phase 0, end phase 1
CIC exon 16, start phase 1, end phase 0
CIC exon 17, start phase 0, end phase 2
CIC exon 18, start phase 2, end phase 1
CIC exon 19, start phase 1, end phase 1
CIC exon 20, start phase 1, end phase 1
SLC12A6 exon 2, start phase 1, end phase 1
YAP1 exon 3, start phase 2, end phase 1
NUTM1 exon 2, start phase 0, end phase 1
NUTM1 exon 3, start phase 1, end phase 2
NUTM1 exon 4, start phase 2, end phase 2
NUTM1 exon 5, start phase 2, end phase 1


In [112]:
valid_pairs = []

for five_prime_name, five_prime_exon_numbers in list(five_prime_genes.items()):
    for five_prime_exon_number in five_prime_exon_numbers:
        five_prime_phase = name_and_exon_start_phase[(five_prime_name, five_prime_exon_number)]

        for three_prime_name, three_prime_exon_numbers in [("NUTM1", NUTM1_start_exons)]:
            for three_prime_exon_number in three_prime_exon_numbers:
                three_prime_phase = name_and_exon_end_phase[(three_prime_name, three_prime_exon_number)]
                if five_prime_phase == three_prime_phase:
                    print("%s exon %d -> %s exon %d" % (
                        five_prime_name, five_prime_exon_number, 
                        three_prime_name, three_prime_exon_number))
                    valid_pairs.append((
                        (five_prime_name, five_prime_exon_number),
                        (three_prime_name, three_prime_exon_number)
                    ))

MGA exon 22 -> NUTM1 exon 2
MGA exon 22 -> NUTM1 exon 5
BRD4 exon 10 -> NUTM1 exon 3
BRD4 exon 10 -> NUTM1 exon 4
BRD4 exon 11 -> NUTM1 exon 2
BRD4 exon 11 -> NUTM1 exon 5
BRD4 exon 12 -> NUTM1 exon 2
BRD4 exon 12 -> NUTM1 exon 5
BRD4 exon 14 -> NUTM1 exon 2
BRD4 exon 14 -> NUTM1 exon 5
BRD4 exon 15 -> NUTM1 exon 2
BRD4 exon 15 -> NUTM1 exon 5
CIC exon 16 -> NUTM1 exon 2
CIC exon 16 -> NUTM1 exon 5
CIC exon 18 -> NUTM1 exon 3
CIC exon 18 -> NUTM1 exon 4
CIC exon 19 -> NUTM1 exon 2
CIC exon 19 -> NUTM1 exon 5
CIC exon 20 -> NUTM1 exon 2
CIC exon 20 -> NUTM1 exon 5
SLC12A6 exon 2 -> NUTM1 exon 2
SLC12A6 exon 2 -> NUTM1 exon 5
YAP1 exon 3 -> NUTM1 exon 3
YAP1 exon 3 -> NUTM1 exon 4


In [138]:
fused_coding_sequences = {}
breakpoint_offset = {}
for (k1, k2) in valid_pairs:
    (p5_name, p5_exon), (p3_name, p3_exon) = k1, k2
    p5_t = name_to_transcript[p5_name]
    p5_full_cds = p5_t.coding_sequence
    p5_cds_length = name_and_exon_to_cds_length[(p5_name, p5_exon)]
    print("Truncating %s CDS (length %d) to exon %d (length %d)" % (
        p5_name,
        len(p5_full_cds),
        p5_exon,
        p5_cds_length,
    ))
    p5_cds = p5_full_cds[:p5_cds_length]
    
    print(len(p5_cds) % 3, name_and_exon_end_phase[k1])
    p3_t = name_to_transcript[p3_name]
    p3_full_cds = p3_t.coding_sequence
    p3_cds_length_before = name_and_exon_to_cds_length[(p3_name, p3_exon - 1)]
    p3_cds_length = len(p3_full_cds) - p3_cds_length_before
    p3_cds = p3_full_cds[p3_cds_length_before:]
    assert len(p3_cds) == p3_cds_length
    print("Starting %s CDS (length %d) at exon %d (length %d)" % (
        p3_name,
        len(p3_full_cds),
        p3_exon,
        len(p3_cds)
    ))
    print(len(p3_cds) % 3, name_and_exon_start_phase[k2])

    combined_cds = p5_cds + p3_cds
    print(k1, k2, len(combined_cds) % 3)
    fused_coding_sequences[(k1, k2)] = combined_cds
    breakpoint_offset[(k1, k2)] = len(p5_cds)
    
    

Truncating MGA CDS (length 8571) to exon 22 (length 7294)
1 1
Starting NUTM1 CDS (length 3483) at exon 2 (length 3477)
0 0
('MGA', 22) ('NUTM1', 2) 1
Truncating MGA CDS (length 8571) to exon 22 (length 7294)
1 1
Starting NUTM1 CDS (length 3483) at exon 5 (length 2545)
1 2
('MGA', 22) ('NUTM1', 5) 2
Truncating BRD4 CDS (length 4089) to exon 10 (length 2047)
1 1
Starting NUTM1 CDS (length 3483) at exon 3 (length 3383)
2 1
('BRD4', 10) ('NUTM1', 3) 0
Truncating BRD4 CDS (length 4089) to exon 10 (length 2047)
1 1
Starting NUTM1 CDS (length 3483) at exon 4 (length 2674)
1 2
('BRD4', 10) ('NUTM1', 4) 2
Truncating BRD4 CDS (length 4089) to exon 11 (length 2158)
1 1
Starting NUTM1 CDS (length 3483) at exon 2 (length 3477)
0 0
('BRD4', 11) ('NUTM1', 2) 1
Truncating BRD4 CDS (length 4089) to exon 11 (length 2158)
1 1
Starting NUTM1 CDS (length 3483) at exon 5 (length 2545)
1 2
('BRD4', 11) ('NUTM1', 5) 2
Truncating BRD4 CDS (length 4089) to exon 12 (length 2211)
0 0
Starting NUTM1 CDS (length 34

In [128]:
def translate(seq):
      
    table = {
        'ATA':'I', 'ATC':'I', 'ATT':'I', 'ATG':'M',
        'ACA':'T', 'ACC':'T', 'ACG':'T', 'ACT':'T',
        'AAC':'N', 'AAT':'N', 'AAA':'K', 'AAG':'K',
        'AGC':'S', 'AGT':'S', 'AGA':'R', 'AGG':'R',                
        'CTA':'L', 'CTC':'L', 'CTG':'L', 'CTT':'L',
        'CCA':'P', 'CCC':'P', 'CCG':'P', 'CCT':'P',
        'CAC':'H', 'CAT':'H', 'CAA':'Q', 'CAG':'Q',
        'CGA':'R', 'CGC':'R', 'CGG':'R', 'CGT':'R',
        'GTA':'V', 'GTC':'V', 'GTG':'V', 'GTT':'V',
        'GCA':'A', 'GCC':'A', 'GCG':'A', 'GCT':'A',
        'GAC':'D', 'GAT':'D', 'GAA':'E', 'GAG':'E',
        'GGA':'G', 'GGC':'G', 'GGG':'G', 'GGT':'G',
        'TCA':'S', 'TCC':'S', 'TCG':'S', 'TCT':'S',
        'TTC':'F', 'TTT':'F', 'TTA':'L', 'TTG':'L',
        'TAC':'Y', 'TAT':'Y', 'TAA':'_', 'TAG':'_',
        'TGC':'C', 'TGT':'C', 'TGA':'_', 'TGG':'W',
    }
    protein = ""
    if len(seq)%3 == 0:
        for i in range(0, len(seq), 3):
            codon = seq[i:i + 3]
            protein+= table[codon]
    return protein

In [130]:
combined_cds

'ATGGATCCCGGGCAGCAGCCGCCGCCTCAACCGGCCCCCCAGGGCCAAGGGCAGCCGCCTTCGCAGCCCCCGCAGGGGCAGGGCCCGCCGTCCGGACCCGGGCAACCGGCACCCGCGGCGACCCAGGCGGCGCCGCAGGCACCCCCCGCCGGGCATCAGATCGTGCACGTCCGCGGGGACTCGGAGACCGACCTGGAGGCGCTCTTCAACGCCGTCATGAACCCCAAGACGGCCAACGTGCCCCAGACCGTGCCCATGAGGCTCCGGAAGCTGCCCGACTCCTTCTTCAAGCCGCCGGAGCCCAAATCCCACTCCCGACAGGCCAGTACTGATGCAGGCACTGCAGGAGCCCTGACTCCACAGCATGTTCGAGCTCATTCCTCTCCAGCTTCTCTGCAGTTGGGAGCTGTTTCTCCTGGGACACTGACCCCCACTGGAGTAGTCTCTGGCCCAGCAGCTACACCCACAGCTCAGCATCTTCGACAGTCTTCTTTTGAGATACCTGATGATGTACCTCTGCCAGCAGGTTGGGAGATGGCAAAGACATCTTCTGGTCAGAGATACTTCTTAAATCACATCGATCAGACAACAACATGGCAGGACCCCAGGAAGGCCATGCTGTCCCAGATGAACGTCACAGCCCCCACCAGTCCACCAGTGCAGCAGAATATGATGAACTCGGCTTCAGCCCAGTGCTTCGTTCCCTGGCCCGGCTGAAGCCCACTATGACCCTGGAGGAGGGACTGCCATTGGCTGTGCAGGAGTGGGAGCACACCAGCAACTTTGACCGGATGATCTTTTATGAGATGGCAGAAAGGTTCATGGAGTTTGAGGCTGAGGAGATGCAGATTCAGAACACACAGCTGATGAATGGGTCTCAGGGCCTGTCTCCTGCAACCCCTTTGAAACTTGATCCTCTAGGGCCCCTGGCCTCTGAGGTTTGCCAGCAGCCAGTGTACATTCCGAAGAAGGCAGCCTCCAAGACACGGGCCCCCCGCC

In [71]:
t.exon_intervals

[(34343315, 34343702),
 (34345942, 34346035),
 (34347969, 34348677),
 (34350704, 34350832),
 (34353736, 34353872),
 (34354446, 34354732),
 (34355021, 34355137),
 (34355488, 34357735)]

In [80]:
2047 / 3


682.3333333333334

In [139]:
3483 / 3

1161.0